In [4]:
import pandas as pd

In [5]:
import matplotlib as mb

In [6]:
from main import distance

In [7]:
df= pd.read_csv(".\kc_house_data.csv")

In [8]:
df.columns

Index(['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15'],
      dtype='object')

Zipcodes:

In [9]:
geo_loc= df.iloc[1:,[0,16,17,18]].copy()
geo_loc

,id,zipcode,lat,long
1,6414100192,98125,47.7210,-122.319
2,5631500400,98028,47.7379,-122.233
3,2487200875,98136,47.5208,-122.393
4,1954400510,98074,47.6168,-122.045
5,7237550310,98053,47.6561,-122.005
...,...,...,...,...
21608,263000018,98103,47.6993,-122.346
21609,6600060120,98146,47.5107,-122.362
21610,1523300141,98144,47.5944,-122.299
21611,291310100,98027,47.5345,-122.069


In [10]:
k=list(geo_loc.zipcode.unique())
k.sort()
for i in k:
    print(i,end=",")


98001,98002,98003,98004,98005,98006,98007,98008,98010,98011,98014,98019,98022,98023,98024,98027,98028,98029,98030,98031,98032,98033,98034,98038,98039,98040,98042,98045,98052,98053,98055,98056,98058,98059,98065,98070,98072,98074,98075,98077,98092,98102,98103,98105,98106,98107,98108,98109,98112,98115,98116,98117,98118,98119,98122,98125,98126,98133,98136,98144,98146,98148,98155,98166,98168,98177,98178,98188,98198,98199,

In [11]:
#pd.set_option('display.max_rows', None)
district_0=geo_loc.loc[geo_loc.zipcode.astype(str).str[2:]=='003',['id','zipcode','lat','long']]
district_0.groupby(['long','lat', 'id']).count()
district_0.agg(['max','min'])



,id,zipcode,lat,long
max,9358400150,98003,47.3554,-122.180
min,100500020,98003,47.2574,-122.334


In [12]:
district_0=geo_loc.loc[geo_loc.zipcode.astype(str).str[2:]=='001',['id','zipcode','lat','long']]
district_0.groupby(['long','lat', 'id']).count()
district_0.agg(['max','min'])



,id,zipcode,lat,long
max,9543000945,98001,47.3575,-122.231
min,128500260,98001,47.2576,-122.294


In [13]:
district_0=geo_loc.loc[geo_loc.zipcode.astype(str).str[3:]=='02',['id','zipcode','lat','long']]
district_0.groupby(['long','lat', 'id']).count()
district_0.agg(['max','min'])


,id,zipcode,lat,long
max,9831200520,98102,47.6507,-122.080
min,1000102,98002,47.2761,-122.385


The first two digits in the zipcode indicate the country and state respectively, the third digit represents the district, and last two digits indicate the city. When we look at gps coordinates we can designate a specified region in which all houses with identical zipcodes fall, this can be discerned from finding the maximal and minimal longitude and latitude values for each zipcode. Furthermore we can remark that these regions do not cross over.

Demand In Housing Per City:


In [14]:
year_built = df[['zipcode', 'price', 'yr_built']].groupby('zipcode').agg(['max', 'min', 'mean', 'median','count']).reset_index()
year_built

zipcode      price                                          yr_built        \
                  max       min          mean     median count      max   min   
0    98001   850000.0  100000.0  2.808047e+05   260000.0   362     2014  1903   
1    98002   389000.0   95000.0  2.342840e+05   235000.0   199     2014  1908   
2    98003   950000.0  128000.0  2.941113e+05   267475.0   280     2014  1927   
3    98004  7060000.0  425000.0  1.356524e+06  1150000.0   317     2014  1908   
4    98005  1960000.0  400000.0  8.102897e+05   765475.0   168     2011  1943   
..     ...        ...       ...           ...        ...   ...      ...   ...   
65   98177  3800000.0  245560.0  6.764194e+05   554000.0   255     2014  1906   
66   98178  1700000.0   90000.0  3.106128e+05   278277.0   262     2014  1905   
67   98188   707000.0  110000.0  2.890783e+05   264000.0   136     2014  1920   
68   98198  1350000.0   95000.0  3.028967e+05   265000.0   280     2014  1905   
69   98199  3200000.0  323000.0  7.921879e+05   689800.0   317     2015  1901   

                               
           mean  median count  
0   1980.679558  1981.0   362  
1   1967.773869  1966.0   199  
2   1976.885714  1975.0   280  
3   1971.470032  1965.0   317  
4   1969.744048  1967.0   168  
..          ...     ...   ...  
65  1960.843137  1956.0   255  
66  1955.232824  1952.0   262  
67  1965.507353  1959.5   136  
68  1966.735714  1962.0   280  
69  1956.526814  1948.0   317  

[70 rows x 11 columns]

In [15]:
year_blt = df[['zipcode', 'price', 'yr_built']].groupby('zipcode').median().reset_index()
median_year_blt=year_blt.copy()
median_year_blt.rename(columns={'price':'median_price', 'yr_built':'med_yr'}, inplace=True)
median_year_blt

,zipcode,median_price,med_yr
0,98001,260000.0,1981.0
1,98002,235000.0,1966.0
2,98003,267475.0,1975.0
3,98004,1150000.0,1965.0
4,98005,765475.0,1967.0
...,...,...,...
65,98177,554000.0,1956.0
66,98178,278277.0,1952.0
67,98188,264000.0,1959.5
68,98198,265000.0,1962.0


In [16]:
best_cities= median_year_blt.loc[median_year_blt['med_yr']>=1995,['zipcode', 'median_price', 'med_yr']]
best_cities

,zipcode,median_price,med_yr
11,98019,401250.0,1997.0
17,98029,575000.0,1999.0
23,98038,342000.0,2000.0
29,98053,635000.0,2003.0
33,98059,435000.0,1997.0
34,98065,500000.0,2004.0
38,98075,739999.0,1998.0


In [17]:
year_blt = df[['zipcode', 'price', 'yr_built']].groupby('zipcode').quantile(0.25).reset_index()
quan_year_blt=year_blt.copy()
quan_year_blt.rename(columns={'price':'quan_price', 'yr_built':'quan75_yr'}, inplace = True)
quan_year_blt

,zipcode,quan_price,quan75_yr
0,98001,215000.0,1966.0
1,98002,199975.0,1953.0
2,98003,229000.0,1966.0
3,98004,845000.0,1953.0
4,98005,637775.0,1959.0
...,...,...,...
65,98177,428500.0,1950.0
66,98178,223250.0,1944.0
67,98188,228750.0,1954.0
68,98198,216375.0,1956.0


In [18]:
quan_year_blt.loc[quan_year_blt['quan75_yr']>=1995,['zipcode', 'quan_price', 'quan75_yr']]

,zipcode,quan_price,quan75_yr
34,98065,424962.5,2000.0


In [19]:
list_cities=list(best_cities.zipcode.values)
list_cities

[98019, 98029, 98038, 98053, 98059, 98065, 98075]

In [20]:

temp=df[['id','zipcode']].groupby(['zipcode']).count().reset_index()
temp.sort_values(by=['id'], ascending=False)
temp['perc'] = (temp['id'] / len(df)) * 100
temp.mean() #Average distribution of houses is around 1.4 percent of total number of houses for each city


zipcode    98077.300000
id           308.757143
perc           1.428571
dtype: float64

In [21]:
temp.median()

zipcode    98067.500000
id           282.500000
perc           1.307084
dtype: float64

In [22]:
percent={}
for i in list_cities:
    k=temp.loc[temp.zipcode==i] 
    val=k.perc
    percent[i]=val
percent

{98019: 11    0.879101
 Name: perc, dtype: float64,
 98029: 17    1.485217
 Name: perc, dtype: float64,
 98038: 23    2.729839
 Name: perc, dtype: float64,
 98053: 29    1.873872
 Name: perc, dtype: float64,
 98059: 33    2.165363
 Name: perc, dtype: float64,
 98065: 34    1.434322
 Name: perc, dtype: float64,
 98075: 38    1.661037
 Name: perc, dtype: float64}

we find 7 cities that have at least half of the housing built in the last 20 years and 1 that has 75% of indicating a substantial surge in demand for housing in that particular city. 

To ascertain our observations we take a look at the distribution of houses across all cities, to make sure the total houses in our list of 7 cities do not have a low distribution in them which would be in contrast to our above conjecture.

Given that the percent mean of distribution is very close to the median, this would indicate that our data is not too skewed, and we can make safe assumptions based on the distributions of houses across the cities.

we find that compared to this mean our, among 7 cities all but one lie either at the mean or above it, indicating they have average distribution. 

Given this analysis, the best cities to invest in would be 98038 with the highest distribution among the 7 and 98065 with the overall highest number of new houses built.


Cost of Construction:

In [39]:
pd.set_option('display.max_rows', None)

In [45]:
cost = df.loc[df['waterfront']==0,['zipcode','price','sqft_living','sqft_lot','floors','waterfront']].groupby('zipcode').mean().round(1).reset_index()
cost

,zipcode,price,sqft_living,sqft_lot,floors,waterfront
0,98001,280804.7,1900.9,14937.5,1.4,0.0
1,98002,234284.0,1627.7,7517.6,1.3,0.0
2,98003,294111.3,1928.9,10603.1,1.3,0.0
3,98004,1338475.0,2886.5,13027.6,1.4,0.0
4,98005,810289.7,2656.8,19928.8,1.3,0.0
5,98006,854148.6,2888.0,13398.7,1.5,0.0
6,98007,617254.0,2182.1,10662.7,1.4,0.0
7,98008,582345.9,2059.8,9989.7,1.2,0.0
8,98010,423666.0,2137.6,55465.3,1.5,0.0
9,98011,490377.1,2253.1,11314.1,1.5,0.0


In [46]:
cost.median()

zipcode         98067.50
price          492001.20
sqft_living      2006.55
sqft_lot        10491.45
floors              1.50
waterfront          0.00
dtype: float64

In [56]:
df.floors.max()
df.price.median()
df.sqft_lot.median()

7618.0

In [58]:
cost.sort_values(by=['price'])
cost_eff = cost.loc[(cost.floors>1.5) & (cost.sqft_living>2006) & (cost.sqft_lot>10491)]
cost_eff.sort_values(by=['sqft_lot']) 

,zipcode,price,sqft_living,sqft_lot,floors,waterfront
28,98052,639790.2,2359.2,11078.8,1.6,0.0
33,98059,493625.3,2403.7,14405.9,1.6,0.0
37,98074,667570.2,2638.7,14578.4,1.7,0.0
34,98065,528003.1,2571.3,17065.2,1.9,0.0
24,98039,2131122.4,3779.9,17304.5,1.6,0.0
38,98075,759323.1,3010.4,19314.2,1.8,0.0
23,98038,366876.1,2147.7,25385.7,1.7,0.0
40,98092,334921.1,2195.7,32494.6,1.6,0.0
15,98027,608356.7,2499.4,32962.0,1.7,0.0
29,98053,678275.7,2620.9,35547.3,1.7,0.0


To roughly analyse the average cost of land and construction in each city we can take a look at the table of houses that aren't at the waterfront, because we observe that waterfront houses tend to skew the dataset too much. By looking at the mean values of prices per the dimensions of land and living area, and the median of those values, we can observe that the houses larger than the median values have roughly similar living and floor specifications. This allows us to make a direct comparison of the total property area and the cost. we see that the second and fifth largest average lot areas have the cheapest average prices. So, to conclude, the cost of land appears to be in general lowest in the cities 98019 and 98092.

Combining our first and latter findings. It seems the best city to invest in is 98019, because the average cost of land is cheaper and it has a blooming real estate market.